In [1]:
import sys
!{sys.executable} -m pip install pymysql

In [2]:
import pandas as pd
import re, pymysql, nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='',db='cprs')
conn!=False

True

In [4]:
data=pd.read_sql(f"SELECT * FROM Shows", conn)
data.index=data.showID

In [5]:
data[:5]

,showID,title,genres,contentType,directors,actors,contentRating,releaseYear,summary,languageID,platformIDs,posterURL,average_rating_from_imdb,count_of_ratings_imdb
showID,,,,,,,,,,,,,,
0,0,Lets Fight Ghost,"Comedy, Drama, Fiction, Horror, Thriller",Series,Tomas Alfredson,"Lina Leandersson, Kåre Hedebrant, Per Ragnar, ...",R,2008,A med student with a supernatural gift tries t...,70,1,https://image.tmdb.org/t/p/w1280/eHKxpDheZPV44...,8.4,5
1,1,HOW TO BUILD A GIRL,Comedy,Movie,Coky Giedroyc,"Cleo, Paddy Considine, Beanie Feldstein, Dónal...",R,2020,"When nerdy Johanna moves to London, things get...",71,1,https://image.tmdb.org/t/p/w1280/svEKCyQmJs3ZR...,0.0,0
2,2,The Con Heartist,Comedy,Movie,Mez Tharatorn,"Kathaleeya McIntosh, Nadech Kugimiya, Pimchano...",Unrated,2020,After her ex boyfriend cons her out of a large...,70,1,https://image.tmdb.org/t/p/w1280/wFJujmOPJTMuH...,0.0,0
3,3,Gleboka woda,Drama,Series,"Magdalena Lazarkiewicz, Kasia Adamik, Olga Cha...","Katarzyna Maciag, Piotr Nowak, Marcin Dorocins...",Unrated,2011,A group of social welfare workers led by their...,72,1,https://image.tmdb.org/t/p/w1280/c5QeX1Z2YXXXB...,0.0,0
4,4,Only a Mother,Drama,Movie,Alf Sjöberg,"Hugo Björne, Eva Dahlbeck, Ulf Palme, Ragnar F...",Unrated,1949,An unhappily married farm worker struggling to...,73,1,https://image.tmdb.org/t/p/w1280/pVYm5Ck5eJcpQ...,6.0,1


In [6]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words_list=stopwords.words('english')
wnl=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dba5153\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dba5153\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def clean_summary(x):
    x=re.sub("\s+", " ", re.sub("[^\w ]", " ", x)).lower()
    x=[word for word in x.split() if not word in stop_words_list]
    x=[wnl.lemmatize(word, pos="v") for word in x]
    return ' '.join(x)

In [8]:
%%time
data['clean_summary']=data.summary.apply(clean_summary)
data.clean_summary

Wall time: 3.67 s


showID
0        med student supernatural gift try cash abiliti...
1        nerdy johanna move london things get hand rein...
2        ex boyfriend con large sum money former bank e...
3        group social welfare workers lead new director...
4        unhappily marry farm worker struggle care chil...
                               ...                        
23526        physicist try help alien cat crash land earth
23527    super sleuth mouse must solve toy maker disapp...
23528       merlin train young orphan destine king england
23529    strong will family struggle establish wild gee...
23532    rock imagination movers disney junior favorite...
Name: clean_summary, Length: 16155, dtype: object

In [9]:
def clean_names(x):
    x=re.split(",\s*", x)[:4]
    x=[re.sub("\s+", "", word.lower()) for word in x]
    return ", ".join(x)

In [10]:
data["clean_genres"]=data.genres.apply(clean_names)
data["clean_actors"]=data.actors.apply(clean_names)
data["clean_directors"]=data.directors.apply(clean_names)

In [11]:
# TF-IDF : Term Frequency and Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [12]:
def makeTfidfDF(data, column="clean_summary"):
    tfidf_data=tfidf.fit_transform(data[column])
    return pd.DataFrame(tfidf_data.todense(), columns=tfidf.get_feature_names(), \
                      index=data.index)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
def generateMatrices(data):
    binary=makeTfidfDF(data)
    return pd.DataFrame(cosine_similarity(binary), index=binary.index, columns=binary.index)

In [16]:
%%time
cosine_similarities=generateMatrices(data)
cosine_similarities.shape

Wall time: 2min 27s


(16155, 16155)

In [17]:
def makeBinaryDFs(data, columnName):
    return pd.DataFrame(mlb.fit_transform(data[columnName].apply(lambda x: re.split(",\s*", x))),
            columns=mlb.classes_, index=data.index)

In [18]:
def generateMatrices2(data): 
    binary=pd.concat([makeBinaryDFs(data, 'genres'), makeBinaryDFs(data, 'directors'), \
              makeBinaryDFs(data, 'actors')], axis=1)
    return pd.DataFrame(cosine_similarity(binary), index=binary.index, columns=binary.index)

In [19]:
%%time
cosine_similarities2=generateMatrices2(data)
cosine_similarities2.shape

Wall time: 5min 33s


(16155, 16155)

In [20]:
def getRecommendations(data, title, index, n):
    results=cosine_similarities[index].sort_values(ascending=False)[:n//2+1]
    results2=cosine_similarities2[index]
    results2=results2[~results2.index.isin(results)].sort_values(ascending=False)[:n//2+1]
    similar=pd.concat([data[data.index.isin(results.index)], data[data.index.isin(results2.index)]])
    return similar[similar.title!=title][:n]

In [21]:
def recommender(showID, n=20):
    results=getRecommendations(data, data.loc[showID, "title"], showID, n)
    return showID, results.showID

In [22]:
similarContent=pd.DataFrame({"contentID":[], "similarContentID":[]})
similarContent.contentID=similarContent.contentID.astype(int)
similarContent.similarContentID=similarContent.similarContentID.astype(int)

In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [24]:
TIMEOUT_SECS=60

In [25]:
%%time
future_to_url, no_data_for={}, []
with ThreadPoolExecutor(max_workers=20) as executor:
    for showID in data.showID:
        future_to_url[executor.submit(recommender, showID)]=showID

    for future in as_completed(future_to_url):
        try:
            result=future.result(timeout=TIMEOUT_SECS)
            showID, showIDs=result[0], result[1]
            similarContent=pd.concat([similarContent, pd.DataFrame({"contentID":[showID]*20, "similarContentID":list(showIDs)})])
        except:
            pass

Wall time: 4min 3s


In [26]:
similarContent.shape

(316760, 2)

In [28]:
similarContent.reset_index(inplace=True)

In [31]:
similarContent.drop(["level_0", "index"], axis=1, inplace=True)
similarContent[:5]

,contentID,similarContentID
0,3,401
1,3,940
2,3,1776
3,3,4309
4,3,8950


In [33]:
similarContent.drop_duplicates(keep="first", inplace=True)
similarContent.shape

(313448, 2)

In [34]:
%%time
try:
    with conn.cursor() as cur:
        for i in similarContent.index:
            showID=similarContent.loc[i, "contentID"]
            similarID=similarContent.loc[i, "similarContentID"]
#             print(showID, similarID)
            cur.execute(f"INSERT INTO SimilarContent VALUES ({showID}, {similarID});")
        conn.commit()
except Exception as error:
    print(error)

Wall time: 1min 5s


In [35]:
conn.close()